In [8]:
import numpy as np
import sys
import pandas as pd

# Set global variables

In [17]:
## create list of experiment details
exp_list = pd.DataFrame(columns=['id','ptype','fctype','fcdate'])

exps = ['b2nn','b2no','b2nq','b2nr','b2ns','b2nt']
ptype = ['pi','incr']*3
fcdate = ['2022-02-14']*2+['2022-02-10']*2+['2022-02-16']*2

for i in range(6):
    exp_list.loc[i*2] = [exps[i],ptype[i],'cf',fcdate[i]]
    exp_list.loc[i*2+1] = [exps[i],ptype[i],'pf',fcdate[i]]

In [18]:
exp_list

,id,ptype,fctype,fcdate
0,b2nn,pi,cf,2022-02-14
1,b2nn,pi,pf,2022-02-14
2,b2no,incr,cf,2022-02-14
3,b2no,incr,pf,2022-02-14
4,b2nq,pi,cf,2022-02-10
5,b2nq,pi,pf,2022-02-10
6,b2nr,incr,cf,2022-02-10
7,b2nr,incr,pf,2022-02-10
8,b2ns,pi,cf,2022-02-16
9,b2ns,pi,pf,2022-02-16


In [11]:
outdir = '/gf3/predict2/AWH012_LEACH_NASTORM/DATA/'

# 1 deg global

In [23]:
# sfc - exp
basepath = outdir+'{exptype}/{ptype}/GLO100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128/228035/228036'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for i,S in exp_list.iterrows():
    
    exp,ptype,fctype,fcdate = S.values
                
    fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
    exp = explist.loc[exptype,ptype]

    reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

    req = 'retrieve,\n'
    req += 'class=uk,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'levtype=sfc,\n'
    if fctype=='pf':
        req += 'number='+number+',\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+',\n'
    req += 'stream=enfo,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=1.0/1.0,\n'
    req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

#     with open(reqpath,'w') as f:

#         f.write(req)

# # just have to change prev pp precip max to 6 hrly precip max & 10fg
# param = '34.128/123.128/136.128/137.128/151.128/164.128/224.228/228.128/228035/228036'

# # sfc - ENS
# exptype = 'MED-R/ENS'
# for fctype in ['pf','cf']:
#     for fcdate in ['2022-02-07']:

#         fclength = (pd.to_datetime('2022-02-22') - pd.to_datetime(fcdate)).days * 24
#         steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
#         exp = '1'

#         reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

#         req = 'retrieve,\n'
#         req += 'class=od,\n'
#         req += 'date='+fcdate+',\n'
#         req += 'expver='+exp+',\n'
#         req += 'levtype=sfc,\n'
#         if fctype=='pf':
#             req += 'number='+number+',\n'
#         req += 'param='+param+',\n'
#         req += 'step='+steps+',\n'
#         req += 'stream=enfo,\n'
#         req += 'time=00:00:00,\n'
#         req += 'type='+fctype+',\n'
#         req += 'format=netcdf,\n'
#         req += 'grid=1.0/1.0,\n'
#         req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)

# sfc - large ensembles
# fcdate = '2021-06-18'
# number = '/'.join(np.arange(1,251).astype(str))
# for exp,expptype in LElist.items():
#     for fctype in ['pf','cf']:

#         fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
#         steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
#         exptype,ptype = expptype.split('_')

#         reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

#         req = 'retrieve,\n'
#         req += 'class=uk,\n'
#         req += 'date='+fcdate+',\n'
#         req += 'expver='+exp+',\n'
#         req += 'levtype=sfc,\n'
#         if fctype=='pf':
#             req += 'number='+number+',\n'
#         req += 'param='+param+',\n'
#         req += 'step='+steps+',\n'
#         req += 'stream=enfo,\n'
#         req += 'time=00:00:00,\n'
#         req += 'type='+fctype+',\n'
#         req += 'format=netcdf,\n'
#         req += 'grid=1.0/1.0,\n'
#         req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

#         with open(reqpath,'w') as f:

#             f.write(req)

In [24]:
# pl - exp
basepath = outdir+'{exptype}/{ptype}/glo100/pl/{fctype}/req_files/{exp}_{fcdate}.req'

params = ['129.128/130.128','129.128','129.128/131/132']
levels= ['850','500','250']

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for exptype in ['cmip6']:
    for ptype in ['pi','incr']:
        for fctype in ['pf','cf']:
            for fcdate in ['2021-06-22']:
                
                fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
                steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
                exp = explist.loc[exptype,ptype]
                
                reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)
                
                req = 'retrieve,\n'
                req += 'class=uk,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'levelist='+levels[0]+',\n'
                req += 'levtype=pl,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param='+params[0]+',\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfo,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'format=netcdf,\n'
                req += 'grid=1.0/1.0,\n'
                req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
                
                for i in np.arange(1,3):
                    req += 'retrieve,\n'
                    req += 'levelist='+levels[i]+',\n'
                    req += 'param='+params[i]+'\n\n'
                
#                 with open(reqpath,'w') as f:

#                     f.write(req)
                
# pl - ENS
exptype = 'ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-20']:

        fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist='+levels[0]+',\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+params[0]+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
        for i in np.arange(1,3):
            req += 'retrieve,\n'
            req += 'levelist='+levels[i]+',\n'
            req += 'param='+params[i]+'\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)

# sfc - large ensembles
fcdate = '2021-06-18'
number = '/'.join(np.arange(1,251).astype(str))
for exp,expptype in LElist.items():
    for fctype in ['pf','cf']:

        fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
        exptype,ptype = expptype.split('_')

        reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=uk,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist='+levels[0]+',\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+params[0]+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

        for i in np.arange(1,3):
            req += 'retrieve,\n'
            req += 'levelist='+levels[i]+',\n'
            req += 'param='+params[i]+'\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

# 0.25 deg EU

In [38]:
# sfc - exp
basepath = outdir+'{exptype}/{ptype}/EU025/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '34.128/123.128/136.128/137.128/151.128/164.128/224.228/228.128/228035/228036' # for ENS
param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128' # for EXP

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_ens = np.arange(6,360,6)
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for i,S in exp_list.iterrows():
    
    param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128' # for EXP
    
    exp,ptype,fctype,fcdate = S.values
    
    exptype='MED-R/EXP'
                
    fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])

    reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

    req = 'retrieve,\n'
    req += 'class=uk,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'levtype=sfc,\n'
    if fctype=='pf':
        req += 'number='+number+',\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+',\n'
    req += 'stream=enfo,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=0.25/0.25,\n'
    req += 'area=E,\n'
    req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
    
    ## 6-hrly data
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength] if x%6==0][1:])
    param='228035/228036'
    req += 'retrieve,\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+'\n\n'

    with open(reqpath,'w') as f:

        f.write(req)
                
# sfc - ENS
# exptype = 'MED-R/ENS'
# for fctype in ['pf','cf']:
#     for fcdate in ['2022-02-{:02d}'.format(x) for x in np.arange(7,17)]:

#         fclength = (pd.to_datetime('2022-02-22') - pd.to_datetime(fcdate)).days * 24
#         steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
#         exp = '1'

#         reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

#         req = 'retrieve,\n'
#         req += 'class=od,\n'
#         req += 'date='+fcdate+',\n'
#         req += 'expver='+exp+',\n'
#         req += 'levtype=sfc,\n'
#         if fctype=='pf':
#             req += 'number='+number+',\n'
#         req += 'param='+param+',\n'
#         req += 'step='+steps+',\n'
#         req += 'stream=enfo,\n'
#         req += 'time=00:00:00,\n'
#         req += 'type='+fctype+',\n'
#         req += 'format=netcdf,\n'
#         req += 'grid=0.25/0.25,\n'
#         req += 'area=E,\n'
#         req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)

# sfc - large ensembles
# fcdate = '2021-06-18'
# number = '/'.join(np.arange(1,251).astype(str))
# for exp,expptype in LElist.items():
#     for fctype in ['pf','cf']:

#         fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
#         steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
#         exptype,ptype = expptype.split('_')

#         reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

#         req = 'retrieve,\n'
#         req += 'class=uk,\n'
#         req += 'date='+fcdate+',\n'
#         req += 'expver='+exp+',\n'
#         req += 'levtype=sfc,\n'
#         if fctype=='pf':
#             req += 'number='+number+',\n'
#         req += 'param='+param+',\n'
#         req += 'step='+steps+',\n'
#         req += 'stream=enfo,\n'
#         req += 'time=00:00:00,\n'
#         req += 'type='+fctype+',\n'
#         req += 'format=netcdf,\n'
#         req += 'grid=0.25/0.25,\n'
#         req += 'area=70/-150/30/-100,\n'
#         req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

#         with open(reqpath,'w') as f:

#             f.write(req)

In [39]:
reqpath

'/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/incr/EU025/sfc/pf/req_files/b2nt_2022-02-16.req'

In [40]:
print(req)

retrieve,
class=uk,
date=2022-02-16,
expver=b2nt,
levtype=sfc,
number=1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50,
param=34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128,
step=0/3/6/9/12/15/18/21/24/27/30/33/36/39/42/45/48/51/54/57/60/63/66/69/72/75/78/81/84/87/90/93/96,
stream=enfo,
time=00:00:00,
type=pf,
format=netcdf,
grid=0.25/0.25,
area=E,
target="b2nt_2022-02-16.nc"

retrieve,
param=228035/228036,
step=6/12/18/24/30/36/42/48/54/60/66/72/78/84/90/96




In [32]:
# pl - exp
basepath = outdir+'{exptype}/{ptype}/pnw025/pl/{fctype}/req_files/{exp}_{fcdate}.req'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for exptype in ['cmip6']:
    for ptype in ['pi','incr']:
        for fctype in ['pf','cf']:
            for fcdate in ['2021-06-22']:
                
                fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
                steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
                exp = explist.loc[exptype,ptype]
                
                reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)
                
                req = 'retrieve,\n'
                req += 'class=uk,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'levelist=850,\n'
                req += 'levtype=pl,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param=130.128,\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfo,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'format=netcdf,\n'
                req += 'grid=0.25/0.25,\n'
                req += 'area=70/-150/30/-100,\n'
                req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
                
#                 with open(reqpath,'w') as f:

#                     f.write(req)
                
# pl - ENS
exptype = 'ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-20']:

        fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist=850,\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param=130.128,\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)
            
            
# pl - large ensembles
fcdate = '2021-06-18'
number = '/'.join(np.arange(1,251).astype(str))
for exp,expptype in LElist.items():
    for fctype in ['pf','cf']:

        fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
        exptype,ptype = expptype.split('_')

        reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=uk,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist=850,\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param=130.128,\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

# 2.5 deg global all pressure levels

In [14]:
# pl - exp
basepath = outdir+'{exptype}/{ptype}/glo250/pl/{fctype}/req_files/{exp}_{fcdate}.req'

param = '129.128/130.128/131/132/133.128/135.128/155.128/157.128'
level = '10/50/100/200/250/500/700/850/925/1000'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for exptype in ['cmip6']:
    for ptype in ['pi','incr']:
        for fctype in ['pf','cf']:
            for fcdate in ['2021-06-22']:
                
                fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
                steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
                exp = explist.loc[exptype,ptype]
                
                reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)
                
                req = 'retrieve,\n'
                req += 'class=uk,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'levelist='+level+',\n'
                req += 'levtype=pl,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param='+param+',\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfo,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'format=netcdf,\n'
                req += 'grid=2.5/2.5,\n'
                req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
                
#                 with open(reqpath,'w') as f:

#                     f.write(req)
                
# pl - ENS
exptype = 'ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-20']:

        fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist='+level+',\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=2.5/2.5,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)

            
# pl - large ensembles
fcdate = '2021-06-18'
number = '/'.join(np.arange(1,251).astype(str))
for exp,expptype in LElist.items():
    for fctype in ['pf','cf']:
        for numbers in np.arange(5):
            number = '/'.join(np.arange(numbers*50+1,(numbers+1)*50+1).astype(str))
            pfname = '_'+number.split('/')[0]+'-'+number.split('/')[-1]

            fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
            steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
            exptype,ptype = expptype.split('_')

            req = 'retrieve,\n'
            req += 'class=uk,\n'
            req += 'date='+fcdate+',\n'
            req += 'expver='+exp+',\n'
            req += 'levelist='+level+',\n'
            req += 'levtype=pl,\n'
            if fctype=='pf':
                req += 'number='+number+',\n'
            else:
                pfname = ''
            req += 'param='+param+',\n'
            req += 'step='+steps+',\n'
            req += 'stream=enfo,\n'
            req += 'time=00:00:00,\n'
            req += 'type='+fctype+',\n'
            req += 'format=netcdf,\n'
            req += 'grid=2.5/2.5,\n'
            req += 'target="'+exp+'_'+fcdate+pfname+'.nc"\n\n'
            
            reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate+pfname)

            with open(reqpath,'w') as f:

                f.write(req)

In [37]:
# sfc - exp
basepath = outdir+'{exptype}/{ptype}/glo250/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = ''

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for exptype in ['cmip6']:
    for ptype in ['pi','incr']:
        for fctype in ['pf','cf']:
            for fcdate in ['2021-06-22']:
                
                fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
                steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
                exp = explist.loc[exptype,ptype]
                
                reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)
                
                req = 'retrieve,\n'
                req += 'class=uk,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'levtype=sfc,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param='+param+',\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfo,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'format=netcdf,\n'
                req += 'grid=2.5/2.5,\n'
                req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
                
#                 with open(reqpath,'w') as f:

#                     f.write(req)
                
# sfc - ENS
exptype = 'ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-20']:

        fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)


# sfc - large ensembles
fcdate = '2021-06-18'
number = '/'.join(np.arange(1,251).astype(str))
for exp,expptype in LElist.items():
    for fctype in ['pf','cf']:

        fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
        exptype,ptype = expptype.split('_')

        reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=uk,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=2.5/2.5,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

#         with open(reqpath,'w') as f:

#             f.write(req)

# M-climate data

Dates required:

- 2021-07-08
- 2021-07-05
- 2021-07-01
- 2021-06-28
- 2021-06-24
- 2021-06-21
- 2021-06-17
- 2021-06-14
- 2021-06-10
- 2021-06-07
- 2021-06-03

## pnw025

In [21]:
# sfc - M-climate
basepath = outdir+'{exptype}/{ptype}/pnw025/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '39.128/137.128/151.128/164.128/167.128/182.128/201.128/228.128'

step_mclim = np.array(np.arange(0,313,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [5]:
# pl - M-climate
basepath = outdir+'{exptype}/{ptype}/pnw025/pl/{fctype}/req_files/{exp}_{fcdate}.req'

param = '130.128'

step_mclim = np.array(np.arange(0,313,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levelist=850,\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

## glo100

In [22]:
# sfc - M-climate
basepath = outdir+'{exptype}/{ptype}/glo100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '151.128/167.128/228.128'

step_mclim = np.array(np.arange(0,313,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [23]:
# pl - M-climate
basepath = outdir+'{exptype}/{ptype}/glo100/pl/{fctype}/req_files/{exp}_{fcdate}.req'

param = '129.128'

step_mclim = np.array(np.arange(0,313,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levelist=500,\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

## glo250
Split into each parameter, split u/v into each number.

In [16]:
# pl - M-climate
basepath = outdir+'{exptype}/{ptype}/glo250/pl/{fctype}/req_files/{exp}_{fcdate}.req'

step_mclim = np.array(np.arange(0,313,12).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'

param_dict = dict(zip(['z','t','u','v','q','w','d'],['129.128','130.128','131','132','133.128','135.128','155.128']))

## get each param individually
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        for pname,param in param_dict.items():
            
            if pname in ['u','v'] and fctype=='pf':
                
                for numbers in number.split('/'):
                
                    steps = '/'.join([str(x) for x in step_mclim])
                    hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
                    exp = '1'

                    reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate+'_'+pname+'_'+numbers)

                    req = 'retrieve,\n'
                    req += 'class=od,\n'
                    req += 'date='+fcdate+',\n'
                    req += 'expver='+exp+',\n'
                    req += 'hdate='+hdate+',\n'
                    req += 'levelist=10/50/100/200/300/400/500/700/850/925/1000,\n'
                    req += 'levtype=pl,\n'
                    req += 'number='+numbers+',\n'
                    req += 'param='+param+',\n'
                    req += 'step='+steps+',\n'
                    req += 'stream=enfh,\n'
                    req += 'time=00:00:00,\n'
                    req += 'type='+fctype+',\n'
                    req += 'grid=2.5/2.5,\n'
                    req += 'target="'+exp+'_'+fcdate+'_'+pname+'_'+numbers+'.grib"\n\n'

                    with open(reqpath,'w') as f:

                        f.write(req)
                
            else:
        
                steps = '/'.join([str(x) for x in step_mclim])
                hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
                exp = '1'

                reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate+'_'+pname)

                req = 'retrieve,\n'
                req += 'class=od,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'hdate='+hdate+',\n'
                req += 'levelist=10/50/100/200/300/400/500/700/850/925/1000,\n'
                req += 'levtype=pl,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param='+param+',\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfh,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'grid=2.5/2.5,\n'
                req += 'target="'+exp+'_'+fcdate+'_'+pname+'.grib"\n\n'

                with open(reqpath,'w') as f:

                    f.write(req)

### pnw025 - JJA ALL hindcasts

In [11]:
# sfc - M-climate
basepath = outdir+'{exptype}/{ptype}/pnw025/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '39.128/137.128/164.128/201.128'

step_mclim = np.array(np.arange(240,355,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim/JJA-ALL'
for fctype in ['pf','cf']:
    for fcdate in ['2021-05-'+x for x in ['20','24','27','31']]+\
                   ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+\
                   ['2021-07-'+x for x in ['01','05','08','12','15','19','22','26','29']]+\
                   ['2021-08-'+x for x in ['02','05','09','12','16']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

# SEAS5 experiment data

In [5]:
SEAS5_exp = pd.Series(index=['heu5','hf1y','hf3x','hp1i','hp1h','hq35','hq36'],data=['ENS','pCO2/pi','pCO2/incr','pIC/pi','pIC/incr','cmip6/pi','cmip6/incr'])

In [6]:
SEAS5_exp

heu5           ENS
hf1y       pCO2/pi
hf3x     pCO2/incr
hp1i        pIC/pi
hp1h      pIC/incr
hq35      cmip6/pi
hq36    cmip6/incr
dtype: object

## glo100

In [9]:
# sfc
basepath = outdir+'SEAS5/{exppath}/glo100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'fc'

for exp,exppath in SEAS5_exp.iteritems():
    for fcdate in ['2021-05-01','2021-06-01']:
        
        reqpath = basepath.format(exppath=exppath,fctype=fctype,exp=exp,fcdate=fcdate)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,6)])
        param = '34.128/151.128/164.128/165.128/166.128/167.128/228.128'

        req = 'retrieve,\n'
        req += 'class=rd,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        req += 'method=1,\n'
        req += 'number='+number+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
        # part II: 24 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,24)])
        param = '39.128/137.128/176.128/177.128/178.128/179.128/182.128'
        
        req += 'retrieve,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+'\n\n'
        
        # part III: 24 hrly variables starting at 24
        steps = '/'.join([str(x) for x in np.arange(24,2977,24)])
        param = '51.128'
        
        req += 'retrieve,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+'\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [8]:
# sfc - Hindcasts (blocks of 5 years)
basepath = outdir+'SEAS5/{exppath}/glo100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

number = {}
number['05-01'] = '/'.join(np.arange(0,51).astype(str))
number['06-01'] = '/'.join(np.arange(0,25).astype(str))

fctype = 'fc'

exp = '1'

for fcyear in np.arange(1981,2021,5):

    startyear = str(fcyear)
    oyears = np.arange(fcyear+1,fcyear+5).astype(str)

    for fcmo in ['05-01','06-01']:

        reqpath = basepath.format(exppath='HC',fctype=fctype,exp=exp,fcdate=startyear+'-'+oyears[-1]+'_'+fcmo)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,6)])
        param = '151.128/167.128/228.128'

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+startyear+'-'+fcmo+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        req += 'method=1,\n'
        req += 'number='+number[fcmo]+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'system=5,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+startyear+'-'+oyears[-1]+'_'+fcmo+'.nc"\n\n'

        ## get other years
        for fcyear in oyears:
            req += 'retrieve,\n'
            req += 'date='+fcyear+'-'+fcmo+'\n\n'

            with open(reqpath,'w') as f:

                f.write(req)

In [10]:
# pl
basepath = outdir+'SEAS5/{exppath}/glo100/pl/{fctype}/req_files/{exp}_{fcdate}.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'fc'

for exp,exppath in SEAS5_exp.iteritems():
    for fcdate in ['2021-05-01','2021-06-01']:
        
        reqpath = basepath.format(exppath=exppath,fctype=fctype,exp=exp,fcdate=fcdate)

        # part I: 850hPa level
        steps = '/'.join([str(x) for x in np.arange(0,2977,12)])
        param = '130.128/129.128'

        req = 'retrieve,\n'
        req += 'class=rd,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist=850,\n'
        req += 'levtype=pl,\n'
        req += 'method=1,\n'
        req += 'number='+number+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
        # part II: 500hPa level
        param = '129.128'
        
        req += 'retrieve,\n'
        req += 'levelist=500,\n'
        req += 'param='+param+'\n\n'
        
        # part III: 300hPa level
        param = '129.128/131/132'
        
        req += 'retrieve,\n'
        req += 'levelist=300,\n'
        req += 'param='+param+'\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [30]:
# pl - Hindcasts (blocks of 1 years)
basepath = outdir+'SEAS5/{exppath}/glo100/pl/{fctype}/req_files/{exp}_{fcdate}.req'

number = {}
number['05-01'] = '/'.join(np.arange(0,51).astype(str))
number['06-01'] = '/'.join(np.arange(0,25).astype(str))

fctype = 'fc'

exp = '1'

for fcyear in np.arange(1981,2021):
    
    fcyear=str(fcyear)

    for fcmo in ['05-01','06-01']:

        reqpath = basepath.format(exppath='HC',fctype=fctype,exp=exp,fcdate=fcyear+'-'+fcmo)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,12)])
        param = '129.128'

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcyear+'-'+fcmo+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist=500,\n'
        req += 'levtype=pl,\n'
        req += 'method=1,\n'
        req += 'number='+number[fcmo]+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'system=5,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcyear+'-'+fcmo+'.nc"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

## pnw025
SEAS5 resolution isn't this high, but for consistency with the retrieved operational data.

In [11]:
# sfc
basepath = outdir+'SEAS5/{exppath}/pnw025/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'fc'

for exp,exppath in SEAS5_exp.iteritems():
    for fcdate in ['2021-05-01','2021-06-01']:
        
        reqpath = basepath.format(exppath=exppath,fctype=fctype,exp=exp,fcdate=fcdate)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,6)])
        param = '164.128/167.128/228.128'

        req = 'retrieve,\n'
        req += 'class=rd,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        req += 'method=1,\n'
        req += 'number='+number+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
        # part II: 24 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,24)])
        param = '39.128/137.128/182.128'
        
        req += 'retrieve,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+'\n\n'
        
        # part III: 24 hrly variables starting at 24
        steps = '/'.join([str(x) for x in np.arange(24,2977,24)])
        param = '51.128/52.128'
        
        req += 'retrieve,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+'\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [7]:
# sfc - Hindcasts (blocks of 5 years); different area due to size constraints
basepath = outdir+'SEAS5/{exppath}/pnw025/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

number = {}
number['05-01'] = '/'.join(np.arange(0,51).astype(str))
number['06-01'] = '/'.join(np.arange(0,25).astype(str))

fctype = 'fc'

exp = '1'

for fcyear in np.arange(1981,2021,5):

    startyear = str(fcyear)
    oyears = np.arange(fcyear+1,fcyear+5).astype(str)

    for fcmo in ['05-01','06-01']:

        reqpath = basepath.format(exppath='HC',fctype=fctype,exp=exp,fcdate=startyear+'-'+oyears[-1]+'_'+fcmo)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,6)])
        param = '164.128/167.128/228.128'

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+startyear+'-'+fcmo+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        req += 'method=1,\n'
        req += 'number='+number[fcmo]+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'system=5,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+startyear+'-'+oyears[-1]+'_'+fcmo+'.nc"\n\n'

        ## get other years
        for fcyear in oyears:
            req += 'retrieve,\n'
            req += 'date='+fcyear+'-'+fcmo+'\n\n'

        # part II: 24 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,24)])
        param = '39.128/137.128/182.128'

        req += 'retrieve,\n'
        req += 'date='+startyear+'-'+fcmo+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+'\n\n'

        ## get other years
        for fcyear in oyears:
            req += 'retrieve,\n'
            req += 'date='+fcyear+'-'+fcmo+'\n\n'

        # part III: 24 hrly variables starting at 24
        steps = '/'.join([str(x) for x in np.arange(24,2977,24)])
        param = '51.128/52.128'

        req += 'retrieve,\n'
        req += 'date='+startyear+'-'+fcmo+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+'\n\n'

        ## get other years
        for fcyear in oyears:
            req += 'retrieve,\n'
            req += 'date='+fcyear+'-'+fcmo+'\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [10]:
# pl
basepath = outdir+'SEAS5/{exppath}/pnw025/pl/{fctype}/req_files/{exp}_{fcdate}.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'fc'

for exp,exppath in SEAS5_exp.iteritems():
    for fcdate in ['2021-05-01','2021-06-01']:
        
        reqpath = basepath.format(exppath=exppath,fctype=fctype,exp=exp,fcdate=fcdate)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,12)])

        req = 'retrieve,\n'
        req += 'class=rd,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist=850,\n'
        req += 'levtype=pl,\n'
        req += 'method=1,\n'
        req += 'number='+number+',\n'
        req += 'origin=ecmf,\n'
        req += 'param=130.128,\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [12]:
# pl - Hindcasts (blocks of 5 years); different area due to size constraints
basepath = outdir+'SEAS5/{exppath}/pnw025/pl/{fctype}/req_files/{exp}_{fcdate}.req'

number = {}
number['05-01'] = '/'.join(np.arange(0,51).astype(str))
number['06-01'] = '/'.join(np.arange(0,25).astype(str))

fctype = 'fc'

exp = '1'

for fcyear in np.arange(1981,2021,5):

    startyear = str(fcyear)
    oyears = np.arange(fcyear+1,fcyear+5).astype(str)

    for fcmo in ['05-01','06-01']:

        reqpath = basepath.format(exppath='HC',fctype=fctype,exp=exp,fcdate=startyear+'-'+oyears[-1]+'_'+fcmo)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,12)])
        param = '130.128'

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+startyear+'-'+fcmo+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist=850,\n'
        req += 'levtype=pl,\n'
        req += 'method=1,\n'
        req += 'number='+number[fcmo]+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'system=5,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+startyear+'-'+oyears[-1]+'_'+fcmo+'.nc"\n\n'

        ## get other years
        for fcyear in oyears:
            req += 'retrieve,\n'
            req += 'date='+fcyear+'-'+fcmo+'\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

## glo250

In [13]:
# pl - will have to split into two to run

varlist = 'quv'

basepath = outdir+'SEAS5/{exppath}/glo250/pl/{fctype}/req_files/{exp}_{fcdate}_'+varlist+'.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'fc'

level = '10/50/100/200/300/500/700/850/925/1000'

steps = '/'.join([str(x) for x in np.arange(0,2977,12)])

if varlist == 'dtz':
    param = '129.128/130.128/155.128'
elif varlist == 'quv':
    param = '131/132/133.128'
else:
    raise NameError('varlist not "dtz" or "quv".')

for exp,exppath in SEAS5_exp.iteritems():
    for fcdate in ['2021-05-01','2021-06-01']:
        
        reqpath = basepath.format(exppath=exppath,fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=rd,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist='+level+',\n'
        req += 'levtype=pl,\n'
        req += 'method=1,\n'
        req += 'number='+number+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=mmsf,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=2.5/2.5,\n'
        req += 'target="'+exp+'_'+fcdate+'_'+varlist+'.nc"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

## Monthly mean SEAS5 data

In [16]:
# sfc
basepath = outdir+'SEAS5/MON/{exppath}/glo100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'fcmean'

for exp,exppath in SEAS5_exp.iteritems():
    for fcdate in ['2021-05-01','2021-06-01']:
        
        reqpath = basepath.format(exppath=exppath,fctype='fc',exp=exp,fcdate=fcdate)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,6)])
        param = '34.128/39.128/51.128/137.128/151.128/164.128/165.128/166.128/167.128/176.172/177.172/178.172/179.172/182.172/228.172'

        req = 'retrieve,\n'
        req += 'class=rd,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'fcmonth=1/2/3/4,\n'
        req += 'levtype=sfc,\n'
        req += 'method=1,\n'
        req += 'number='+number+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'stream=msmm,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [55]:
# sfc - hindcast means
basepath = outdir+'SEAS5/MON/{exppath}/glo100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'hcmean'

exp = '1'
exppath='HC'

for fcdate in ['2021-05-01','2021-06-01']:

    reqpath = basepath.format(exppath=exppath,fctype='fc',exp=exp,fcdate=fcdate)

    # part I: 6 hrly variables
    steps = '/'.join([str(x) for x in np.arange(0,2977,6)])
    param = '34.128/39.128/51.128/137.128/151.128/164.128/165.128/166.128/167.128/176.172/177.172/178.172/179.172/182.172/228.172'

    req = 'retrieve,\n'
    req += 'class=od,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'fcmonth=1/2/3/4,\n'
    req += 'levtype=sfc,\n'
    req += 'method=1,\n'
    req += 'origin=ecmf,\n'
    req += 'param='+param+',\n'
    req += 'stream=msmm,\n'
    req += 'system=5,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=1.0/1.0,\n'
    req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

    with open(reqpath,'w') as f:

        f.write(req)

In [17]:
# pl
basepath = outdir+'SEAS5/MON/{exppath}/glo250/pl/{fctype}/req_files/{exp}_{fcdate}.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'fcmean'

for exp,exppath in SEAS5_exp.iteritems():
    for fcdate in ['2021-05-01','2021-06-01']:
        
        reqpath = basepath.format(exppath=exppath,fctype='fc',exp=exp,fcdate=fcdate)

        # part I: 6 hrly variables
        steps = '/'.join([str(x) for x in np.arange(0,2977,6)])
        param = '129.128/130.128/131/132/133.128/155.128'

        req = 'retrieve,\n'
        req += 'class=rd,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'fcmonth=1/2/3/4,\n'
        req += 'levelist=10/50/100/200/250/300/400/500/700/850/925/1000,\n'
        req += 'levtype=pl,\n'
        req += 'method=1,\n'
        req += 'number='+number+',\n'
        req += 'origin=ecmf,\n'
        req += 'param='+param+',\n'
        req += 'stream=msmm,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=2.5/2.5,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [57]:
# pl - hindcast means
basepath = outdir+'SEAS5/MON/{exppath}/glo250/pl/{fctype}/req_files/{exp}_{fcdate}.req'

number = '/'.join(np.arange(0,51).astype(str))

fctype = 'hcmean'

exp = '1'
exppath='HC'

for fcdate in ['2021-05-01','2021-06-01']:

    reqpath = basepath.format(exppath=exppath,fctype='fc',exp=exp,fcdate=fcdate)

    # part I: 6 hrly variables
    steps = '/'.join([str(x) for x in np.arange(0,2977,6)])
    param = '129.128/130.128/131/132/133.128/155.128'

    req = 'retrieve,\n'
    req += 'class=od,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'fcmonth=1/2/3/4,\n'
    req += 'levelist=10/50/100/200/250/300/400/500/700/850/925/1000,\n'
    req += 'levtype=pl,\n'
    req += 'method=1,\n'
    req += 'origin=ecmf,\n'
    req += 'param='+param+',\n'
    req += 'stream=msmm,\n'
    req += 'system=5,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=2.5/2.5,\n'
    req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

    with open(reqpath,'w') as f:

        f.write(req)